In [1]:
import pandas as pd
import numpy as np
import ipaddress
import time
import matplotlib.pyplot as plt

import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import roc_curve, auc
from sklearn.utils import class_weight
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [ ]:
dataset_path1 = 'dataset/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv'
dataset_path2 = 'dataset/Friday-WorkingHours-Morning.pcap_ISCX.csv'
dataset_path3 = 'dataset/Monday-WorkingHours.pcap_ISCX.csv'
dataset_path4 = 'dataset/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv'
dataset_path5 = 'dataset/Tuesday-WorkingHours.pcap_ISCX.csv'
dataset_path6 = 'dataset/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv'

data1 = pd.read_csv(dataset_path1)
data2 = pd.read_csv(dataset_path2)
data3 = pd.read_csv(dataset_path3)
data4 = pd.read_csv(dataset_path4)
data5 = pd.read_csv(dataset_path5)
# data6 = pd.read_csv(dataset_path6)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x96 in position 22398: invalid start byte

In [ ]:
# IP 주소를 정수로 변환하는 함수
def convert_ip2int(ip):
    try:
        return int(ipaddress.ip_address(ip))
    except ValueError:
        return 0

# 데이터 전처리 및 시계열 데이터 생성 함수
def preprocess_and_create_sequences(data, time_window='1T', time_steps=10):
    df = data.copy()
    # 컬럼 이름 정리 및 결측치 처리
    df.columns = df.columns.str.strip().str.replace('[ /]', '_', regex=True)
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.dropna(inplace=True)

    # 타임스탬프 변환 및 IP 주소 정수 변환
    df['Timestamp'] = pd.to_datetime(df['Timestamp'], errors='coerce')
    df.dropna(subset=['Timestamp'], inplace=True)
    
    df['Source_IP'] = df['Source_IP'].astype(str).apply(convert_ip2int)
    df['Destination_IP'] = df['Destination_IP'].astype(str).apply(convert_ip2int)
    df.drop(columns=['Flow_ID'], inplace=True)

    # 라벨 변환 및 집계
    df.set_index('Timestamp', inplace=True)
    df['Label'] = np.where(df['Label'] == 'BENIGN', 0, 1)

    # 집계 함수 정의
    agg_funcs = {col: 'mean' for col in df.columns if col not in ['Label', 'Source_IP', 'Destination_IP']}
    agg_funcs['Total_Fwd_Packets'] = 'sum' # 분 단위 패킷의 합
    agg_funcs['Total_Backward_Packets'] = 'sum' # 분 단위 패킷의 합
    agg_funcs['Total_Length_of_Fwd_Packets'] = 'sum' # 분 단위 패킷의 합
    agg_funcs['Label'] = 'max' # 해당 분에 하나라도 공격이 있으면 1로 표시
    agg_funcs['Source_IP'] = 'nunique' # 고유한 Source IP의 개수
    agg_funcs['Destination_IP'] = 'nunique' # 고유한 Destination IP의 개수

    # time window(1분)에 따라 데이터 집계
    df_agg = df.resample(time_window).agg(agg_funcs)
    df_agg.fillna(0, inplace=True)

    X_agg = df_agg.drop('Label', axis=1).values
    y_agg = df_agg['Label'].values
    
    # 10분 단위의 시계열 데이터 생성
    X_seq, y_seq = [], []
    for i in range(len(X_agg) - time_steps):
        X_seq.append(X_agg[i:(i + time_steps)])
        y_seq.append(y_agg[i + time_steps - 1])
        
    return np.array(X_seq), np.array(y_seq)

# 하이퍼파라미터 정의
TIME_STEPS = 10
TIME_WINDOW = '1T'

print("데이터 전처리 및 시계열 데이터 생성 시작...")
X_data1_seq, y_data1_seq = preprocess_and_create_sequences(data1, time_window=TIME_WINDOW, time_steps=TIME_STEPS)
X_data2_seq, y_data2_seq = preprocess_and_create_sequences(data2, time_window=TIME_WINDOW, time_steps=TIME_STEPS)
X_data3_seq, y_data3_seq = preprocess_and_create_sequences(data3, time_window=TIME_WINDOW, time_steps=TIME_STEPS)
X_data4_seq, y_data4_seq = preprocess_and_create_sequences(data4, time_window=TIME_WINDOW, time_steps=TIME_STEPS)
X_data5_seq, y_data5_seq = preprocess_and_create_sequences(data5, time_window=TIME_WINDOW, time_steps=TIME_STEPS)
# X_data6_seq, y_data6_seq = preprocess_and_create_sequences(data6, time_window=TIME_WINDOW, time_steps=TIME_STEPS)

print("데이터 전처리 및 시계열 데이터 생성 완료.")

print(f"Train 데이터 형태: {X_data1_seq.shape}")
print(f"Test 데이터 형태: {X_data2_seq.shape}")
print(f"Validation 데이터 형태: {X_data3_seq.shape}")
print(f"Additional Train 데이터 형태: {X_data4_seq.shape}")
print(f"Additional Test 데이터 형태: {X_data5_seq.shape}")
# print(f"Additional Validation 데이터 형태: {X_data6_seq.shape}")

print("-" * 50)



데이터 전처리 및 시계열 데이터 생성 시작...


C:\Users\bell1\AppData\Local\Temp\ipykernel_10704\3958384918.py:54: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df_agg = df.resample(time_window).agg(agg_funcs)
C:\Users\bell1\AppData\Local\Temp\ipykernel_10704\3958384918.py:54: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df_agg = df.resample(time_window).agg(agg_funcs)


데이터 전처리 및 시계열 데이터 생성 완료.
Train 데이터 형태: (710, 10, 82)
Test 데이터 형태: (83, 10, 82)
--------------------------------------------------
